# Split the synthetic and experimental datasets

In [15]:
# Defines imports
import numpy as np
import keras
import matplotlib.pyplot as plt
import os
import random
import pickle 
from sklearn.preprocessing import MinMaxScaler
import scipy.io as scio

In [2]:
# Defines constants
_DATA_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/data/imagenet_new'
_PERCENTAGE_TRAIN = 0.8
_PERCENTAGE_VAL = 0.1
_NUM_SAMPLES = 400
_SNRS_DB = [30,40,50,90]
_NUM_NOISE_LEVELS = len(_SNRS_DB)
_ALL_SYNTHETIC_DATASETS = ['dog','goldfish','hotpot','panda','sealion','terrier']
_COMBINATIONS = [['dog'],
    ['dog','sealion'],
    ['dog','panda'],
    ['dog','sealion','panda'],
    ['dog','goldfish','hotpot','panda','sealion','terrier'],
    ['goldfish','hotpot','panda','sealion','terrier']]
# Random seed to have replicable results
_RANDOM_STATE = 20181970
_OUT_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/datasets'

## Synthetic dataset

In [21]:
if not(os.path.exists(_OUT_FOLDER)):
    os.mkdir(_OUT_FOLDER)
synthetic_out_folder = os.path.join(_OUT_FOLDER,'synthetic')
experimental_out_folder = os.path.join(_OUT_FOLDER,'experimental')
if not(os.path.exists(synthetic_out_folder)):
    os.mkdir(synthetic_out_folder)
if not(os.path.exists(experimental_out_folder)):
    os.mkdir(experimental_out_folder)

In [22]:
all_data = {}

for basename in _ALL_SYNTHETIC_DATASETS:
    for noise_level in _SNRS_DB:
        all_data[basename+'_'+str(noise_level)+'dB']={'groundtruth_file': os.path.join(_DATA_FOLDER, basename+'_groundtruth.mat'),
                        'mt_file': os.path.join(_DATA_FOLDER,basename+'_MT_'+str(noise_level)+'dB.mat'),
                        'diffusion_file': os.path.join(_DATA_FOLDER,basename+'_diffusion_'+str(noise_level)+'dB_normalized.mat')}

for key in all_data.keys():
    print(key)

dog_30dB
dog_40dB
dog_50dB
dog_90dB
goldfish_30dB
goldfish_40dB
goldfish_50dB
goldfish_90dB
hotpot_30dB
hotpot_40dB
hotpot_50dB
hotpot_90dB
panda_30dB
panda_40dB
panda_50dB
panda_90dB
sealion_30dB
sealion_40dB
sealion_50dB
sealion_90dB
terrier_30dB
terrier_40dB
terrier_50dB
terrier_90dB


### Load and format groundtruth

In [13]:
def load_groundtruth(groundtruth_file):
    grountruth_contents = scio.loadmat(groundtruth_file)
    img_fitted_params_groundtruth = np.concatenate((grountruth_contents['axon_fit_params'],grountruth_contents['gratio_fit_params']),axis=2)
    brain_mask = grountruth_contents['mask']
    img_gratio_samples = grountruth_contents['gratio_samples']
    img_axon_samples = grountruth_contents['axon_samples']
    img_fractions = grountruth_contents['fractions_groundtruth']
    height, width = brain_mask.shape
    list_indices = []
    gratio_samples = []
    diameter_samples = []
    fractions_samples = []
    # groundtruth parameters that come from fitting the histology samples directly
    # order is alpha, beta (matlab parametrization of Gamma)
    fitted_params = []
    for idx_y in range(height):
        for idx_x in range(width):
            if brain_mask[idx_y,idx_x]:
                # saves the index position in the image for future reconstruction
                list_indices.append(np.array([idx_y,idx_x]))
                # extracts samples and fractions
                gratio_samples.append(np.squeeze(img_gratio_samples[idx_y,idx_x,:]))
                diameter_samples.append(np.squeeze(img_axon_samples[idx_y,idx_x,:]))
                fractions_samples.append(np.squeeze(img_fractions[idx_y,idx_x,:]))
                fitted_params.append(np.squeeze(img_fitted_params_groundtruth[idx_y,idx_x,:]))
    gratio_samples = np.array(gratio_samples)
    diameter_samples = np.array(diameter_samples)
    fractions_samples = np.array(fractions_samples)
    fitted_params = np.array(fitted_params)
    num_voxels = len(list_indices)
    print(f'{num_voxels} voxels available')
    return list_indices, gratio_samples, diameter_samples, fractions_samples, fitted_params

def load_mri(diff_file, mt_files, list_indices):
    # Loads the MT images
    MT_data = scio.loadmat(mt_files)
    img_mt = MT_data['simulated_MTs']
    # Loads the diffusion images
    Diff_data = scio.loadmat(diff_file)
    img_diff = Diff_data['diffusion_normalized']
    diff_samples = []
    mt_samples = []
    for valid_index in list_indices:
        idx_y = valid_index[0]
        idx_x = valid_index[1]
        diff_samples.append(np.squeeze(img_diff[idx_y,idx_x,:]))
        mt_samples.append(np.squeeze(img_mt[idx_y,idx_x,:]))
    diff_samples = np.array(diff_samples)
    mt_samples = np.array(mt_samples)
    x_all = np.concatenate((diff_samples,mt_samples),axis=1)
    num_features = x_all.shape[1]
    print(f'{num_features} MRI features')
    return x_all

In [28]:
for key, info in all_data.items():
    print(key)
    list_indices, gratio_samples, diameter_samples, fractions_samples, fitted_params = load_groundtruth(info['groundtruth_file'])
    info['gratio_samples'] = gratio_samples[:,:_NUM_SAMPLES]
    info['diameter_samples'] = diameter_samples[:,:_NUM_SAMPLES]
    info['fractions_samples'] = fractions_samples
    info['fitted_params'] = fitted_params
    # Loads all noise levels
    x_all = load_mri(info['diffusion_file'], info['mt_file'], list_indices)
    info['mri_data'] = x_all

dog_30dB
1095 voxels available
86 MRI features
dog_40dB
1095 voxels available
86 MRI features
dog_50dB
1095 voxels available
86 MRI features
dog_90dB
1095 voxels available
86 MRI features
goldfish_30dB
7755 voxels available
86 MRI features
goldfish_40dB
7755 voxels available
86 MRI features
goldfish_50dB
7755 voxels available
86 MRI features
goldfish_90dB
7755 voxels available
86 MRI features
hotpot_30dB
7990 voxels available
86 MRI features
hotpot_40dB
7990 voxels available
86 MRI features
hotpot_50dB
7990 voxels available
86 MRI features
hotpot_90dB
7990 voxels available
86 MRI features
panda_30dB
7877 voxels available
86 MRI features
panda_40dB
7877 voxels available
86 MRI features
panda_50dB
7877 voxels available
86 MRI features
panda_90dB
7877 voxels available
86 MRI features
sealion_30dB
7990 voxels available
86 MRI features
sealion_40dB
7990 voxels available
86 MRI features
sealion_50dB
7990 voxels available
86 MRI features
sealion_90dB
7990 voxels available
86 MRI features
terr

### Divide in training and testing

In [30]:
def concatenate_data(data_struct,sel_keys,field):
    list_data = []
    for key, item in data_struct.items():
        if key in sel_keys:
            list_data.append(item[field])
    return np.concatenate(list_data,axis=0)

In [32]:
# Divides the datasets into train, validation and testing.
splits = ['train','val','test']
for combination_basenames in _COMBINATIONS:
    
    # Initializes data structures
    data_save = {}
    indices = {}
    for split in splits:
        data_save[split] = {}
        indices[split] = []
        
    all_names = []
    name_dataset = '_'.join(combination_basenames)
    for basename in combination_basenames:
        for noise_level in _SNRS_DB:
            all_names.append(basename+'_'+str(noise_level)+'dB')

    x_all = concatenate_data(all_data,all_names, 'mri_data')
    diameter_samples = concatenate_data(all_data, all_names, 'diameter_samples')
    gratio_samples = concatenate_data(all_data, all_names, 'gratio_samples')
    fractions_samples = concatenate_data(all_data, all_names, 'fractions_samples')
    fitted_params = concatenate_data(all_data, all_names, 'fitted_params')
    num_features = x_all.shape[1]
    num_voxels = x_all.shape[0]
    
    num_voxels_per_snr = int(num_voxels/len(all_names))
    shuffled_indices = list(range(num_voxels_per_snr))
    random.Random(_RANDOM_STATE).shuffle(shuffled_indices)
    num_train = int(_PERCENTAGE_TRAIN*num_voxels_per_snr)
    num_val = int(_PERCENTAGE_VAL*num_voxels_per_snr)
    num_test = num_voxels_per_snr - num_train - num_val

    indices_train = []
    indices_val = []
    indices_test = []

    for ii in range(len(all_names)):
        start = ii*num_voxels_per_snr
        indices['train'] += list(start+np.array(shuffled_indices[:num_train]))
        indices['val'] += list(start+np.array(shuffled_indices[num_train:(num_train+num_val)]))
        indices['test'] += list(start+np.array(shuffled_indices[(num_train+num_val):]))

    print(f'Using {num_train} for training')
    print(f'Using {num_val} for validation')
    print(f'Using {num_test} for testing')
    
    # Scaler of features
    x_train = x_all[indices['train'],:]
    scaler = MinMaxScaler()
    scaler.fit_transform(x_train)

    print(f'===={name_dataset}=====')
    for split in splits:
        split_indices = indices[split]
        data_save[split]['scaler'] = scaler
        data_save[split]['indices'] = split_indices
        data_save[split]['gratio_samples'] = gratio_samples[split_indices,:]
        data_save[split]['diameter_samples'] = diameter_samples[split_indices,:]
        data_save[split]['fractions_samples'] = fractions_samples[split_indices,:]
        data_save[split]['params_gt'] = fitted_params[split_indices,:]
        x = x_all[split_indices,:]
        print(split+': '+str(x.shape))
        x_scaled = scaler.transform(x)
        print(f'x_{split} min: {x_scaled.min()}')
        print(f'x_{split} max: {x_scaled.max()}')
        data_save[split]['x_unnormalized'] = x
        data_save[split]['x_scaled'] = x_scaled
    with open(os.path.join(synthetic_out_folder,f'{name_dataset}.pkl'), 'wb') as file:
        pickle.dump(data_save, file)

Using 876 for training
Using 109 for validation
Using 110 for testing
====dog=====
train: (3504, 86)
x_train min: 0.0
x_train max: 1.0000000000000004
val: (436, 86)
x_val min: 0.0
x_val max: 1.0000000000000004
test: (440, 86)
x_test min: 0.0
x_test max: 1.0000000000000004
Using 3633 for training
Using 454 for validation
Using 455 for testing
====dog_sealion=====
train: (29064, 86)
x_train min: 0.0
x_train max: 1.0000000000000004
val: (3632, 86)
x_val min: 0.0
x_val max: 1.0000000000000004
test: (3640, 86)
x_test min: 0.0
x_test max: 1.0000000000000004
Using 3588 for training
Using 448 for validation
Using 450 for testing
====dog_panda=====
train: (28704, 86)
x_train min: 0.0
x_train max: 1.0000000000000004
val: (3584, 86)
x_val min: 0.0
x_val max: 1.0000000000000004
test: (3600, 86)
x_test min: 0.0
x_test max: 1.0000000000000004
Using 4523 for training
Using 565 for validation
Using 566 for testing
====dog_sealion_panda=====
train: (54276, 86)
x_train min: 0.0
x_train max: 1.0000000000

## Experimental dataset

In [18]:
_DATA_FOLDER = '/home/gustavo/Gdrive/Stanford/Lab/ODF_prediction/data'
_MOUSE_DATA = [os.path.join(_DATA_FOLDER,'mouse_seizure_data_genu.mat'),
os.path.join(_DATA_FOLDER,'mouse_seizure_data_body.mat'),
os.path.join(_DATA_FOLDER,'mouse_seizure_data_splenium.mat')]

In [19]:
x_mouse = []
axon_fit_params_mouse = []
axon_samples_mouse = []
fractions_mouse = []
gratio_fit_params_mouse = []
gratio_samples_mouse = []
groundtruth_params = []
for name in _MOUSE_DATA:
    mouse_data = scio.loadmat(name)
    x_mouse.append(mouse_data['MRI_inputs'])
    axon_fit_params_mouse.append(mouse_data['axon_fit_params'])
    axon_samples_mouse.append(mouse_data['axon_samples'])
    fractions_mouse.append(mouse_data['fractions_groundtruth'])
    gratio_fit_params_mouse.append(mouse_data['gratio_fit_params'])
    gratio_samples_mouse.append(mouse_data['gratio_samples'])

x_mouse = np.concatenate(x_mouse,axis=0)
axon_fit_params_mouse = np.concatenate(axon_fit_params_mouse,axis=0)
axon_samples_mouse = np.concatenate(axon_samples_mouse,axis=0)
fractions_mouse = np.concatenate(fractions_mouse,axis=0)
gratio_fit_params_mouse = np.concatenate(gratio_fit_params_mouse,axis=0)
gratio_samples_mouse = np.concatenate(gratio_samples_mouse,axis=0)   
groundtruth_params=np.concatenate((axon_fit_params_mouse,gratio_fit_params_mouse), axis = 1)

In [22]:
# Got separation from random in matlab, we maintain 1 HET and 1 WT in each validation and testing
ind_train_base = [1,8,6,2,0]
ind_val_base = [3,4]
ind_test_base = [5,7]

ind_train = []
ind_val = []
ind_test = []
num_voxels_per_snr = 9
for ii in range(len(_MOUSE_DATA)):
    start = ii*num_voxels_per_snr
    ind_train += list(start+np.array(ind_train_base))
    ind_val += list(start+np.array(ind_val_base))
    ind_test += list(start+np.array(ind_test_base))

# Remove last sample of training set (there was a problem with that mouse in the splenium)
ind_train.pop()    
splits = ['train','val','test']

data_mouse = {}
indices = {}
for split in splits:
    data_mouse[split] = {}
indices['train'] = np.array(ind_train)
indices['val'] = np.array(ind_val)
indices['test'] = np.array(ind_test)

x_mouse_train = x_mouse[indices['train'],:]
scaler = MinMaxScaler()
scaler.fit_transform(x_mouse_train)

for split in splits:
    split_indices = indices[split]
    data_mouse[split]['scaler'] = scaler
    data_mouse[split]['indices'] = split_indices
    data_mouse[split]['gratio_samples'] = gratio_samples_mouse[split_indices,:]
    data_mouse[split]['diameter_samples'] = axon_samples_mouse[split_indices,:]
    data_mouse[split]['fractions_samples'] = fractions_mouse[split_indices,:]
    data_mouse[split]['params_gt'] = groundtruth_params[split_indices,:]
    x = x_mouse[split_indices,:]
    x_scaled = scaler.transform(x)
    print(split+': '+str(x.shape))
    print(f'x_{split} min: {x_scaled.min()}')
    print(f'x_{split} max: {x_scaled.max()}')
    data_mouse[split]['x_unnormalized'] = x
    data_mouse[split]['x_scaled'] = x_scaled
    with open(os.path.join(experimental_out_folder,'mouse_3regions.pkl'), 'wb') as file:
        pickle.dump(data_mouse, file)

train: (14, 86)
x_train min: 0.0
x_train max: 1.0000000000000004
val: (6, 86)
x_val min: -0.6289138429481089
x_val max: 1.8381362372257237
test: (6, 86)
x_test min: -0.9860273451163553
x_test max: 1.3754582067145866
